In [3]:
# Import required libraries
import trimesh
import numpy as np
import os
import csv

# Path to the folder containing STL files
folder_path = 'dataset- check'

# Output CSV file
output_csv = 'model_flat_freeform_results.csv'

# Thresholds for classification
VARIANCE_RATIO_THRESHOLD = 0.1  # PCA variance ratio for flatness
THICKNESS_RATIO_THRESHOLD = 0.1 # Bounding box thickness ratio for flatness
OVERHANG_ANGLE_THRESHOLD = 30   # Degrees, for overhang detection

def is_flat_by_pca(mesh):
    # Get vertices
    points = mesh.vertices
    # Center the points
    centered = points - points.mean(axis=0)
    # PCA: covariance and eigenvalues
    cov = np.cov(centered.T)
    eigvals, _ = np.linalg.eigh(cov)
    eigvals = np.sort(eigvals)[::-1]
    # Ratio of smallest to largest eigenvalue
    ratio = eigvals[-1] / eigvals[0]
    return ratio < VARIANCE_RATIO_THRESHOLD

def is_flat_by_bbox(mesh):
    # Get bounding box extents
    extents = mesh.bounding_box.extents
    thickness = np.min(extents)
    max_dim = np.max(extents)
    ratio = thickness / max_dim
    return ratio < THICKNESS_RATIO_THRESHOLD

def is_flat_by_overhang(mesh):
    # Calculate face normals
    normals = mesh.face_normals
    # Reference axis (Z up)
    z_axis = np.array([0, 0, 1])
    # Angles between normals and Z axis
    angles = np.degrees(np.arccos(np.clip(normals @ z_axis, -1, 1)))
    # Count faces with large overhang angles
    overhang_faces = np.sum(angles > OVERHANG_ANGLE_THRESHOLD)
    # If most faces are not overhanging, it's flat
    return overhang_faces < (0.2 * len(angles))

def classify_model(mesh):
    # Use all three methods
    flat_pca = is_flat_by_pca(mesh)
    flat_bbox = is_flat_by_bbox(mesh)
    flat_overhang = is_flat_by_overhang(mesh)
    # If at least two methods say flat, classify as flat
    flat_votes = sum([flat_pca, flat_bbox, flat_overhang])
    return 'flat' if flat_votes >= 2 else 'free form'

# List STL files in the folder
files = [f for f in os.listdir(folder_path) if f.lower().endswith('.stl')]

results = []
for filename in files:
    filepath = os.path.join(folder_path, filename)
    try:
        mesh = trimesh.load(filepath, force='mesh')
        label = classify_model(mesh)
        results.append([filename, label])
        print(f"{filename}: {label}")
    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Write results to CSV
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['filename', 'classification'])
    writer.writerows(results)
print(f"Results written to {output_csv}")

103742.stl: flat
103815.stl: flat
103817.stl: free form
103821.stl: free form
103824.stl: free form
103825.stl: free form
103826.stl: free form
104187.stl: free form
104188.stl: free form
104290.stl: flat
104294.stl: flat
104301.stl: flat
104400.stl: free form
104401.stl: free form
103826.stl: free form
104187.stl: free form
104188.stl: free form
104290.stl: flat
104294.stl: flat
104301.stl: flat
104400.stl: free form
104401.stl: free form
104402.stl: free form
104403.stl: free form
104404.stl: free form
104421.stl: flat
104431.stl: free form
104432.stl: flat
104402.stl: free form
104403.stl: free form
104404.stl: free form
104421.stl: flat
104431.stl: free form
104432.stl: flat
104442.stl: free form
104445.stl: free form
104509.stl: free form
104510.stl: free form
104511.stl: flat
104512.stl: flat
104513.stl: flat
104442.stl: free form
104445.stl: free form
104509.stl: free form
104510.stl: free form
104511.stl: flat
104512.stl: flat
104513.stl: flat
104559.stl: free form
104563.stl: 

**Model Size classification**

In [1]:
# Install numpy-stl if needed
!pip install numpy-stl

import os
import csv
from stl import mesh

# Folder containing STL files
folder = 'dataset- check'

# Output CSV file
output_csv = 'small_big_results.csv'

# Thresholds in mm
SMALL = 50
BIG = 150

# Prepare results list
results = []

# Loop through all STL files in the folder
for filename in os.listdir(folder):
    if filename.lower().endswith('.stl'):
        filepath = os.path.join(folder, filename)
        try:
            model = mesh.Mesh.from_file(filepath)
            min_coords = model.points.min(axis=0)
            max_coords = model.points.max(axis=0)
            size = max_coords - min_coords
            max_dimension = size.max()
            if max_dimension <= SMALL:
                category = 'SMALL'
            elif max_dimension > BIG:
                category = 'BIG'
            else:
                category = 'MEDIUM'
            results.append([filename, f"{max_dimension:.2f}", category])
        except Exception as e:
            results.append([filename, 'ERROR', str(e)])

# Write results to CSV
with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'max_dimension_mm', 'category'])
    writer.writerows(results)

print(f"Results saved to {output_csv}")

Results saved to small_big_results.csv
